# Building POS Tagger using Keras Library without considering PAD values

We are going to work with Twitter *Hindi-English* code mixed tweets. For the purpose of data, we have 1981 tweets which are in **conll** format. They are tagged manually with the *language* and *POS*

#### Load Dependencies

In [2]:
import keras
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding # new!
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D

from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint # new! 
from sklearn.model_selection import train_test_split
import os # new! 
from sklearn.metrics import roc_auc_score, roc_curve # new!
import pandas as pd
import matplotlib.pyplot as plt # new!
%matplotlib inline

In [3]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [4]:
# function to define arr with (word, lang, tag)
def make_arr(f1):
    twitter_file = open(f1, "r")
    sentences = []
    sent = []
    for line in twitter_file:
        temp = line.split('\t')
        
        if temp[0] == '\n':
            sentences.append(sent)
            sent = []
            continue

        check = list(temp[2])
        if '\n' in check:
            check.remove('\n')

        temp[2] = ''.join(check)
        sent.append((temp[0], temp[1], temp[2]))


    return sentences

#### Load data 

In [5]:
tagged_sentences = make_arr("Twitter_file.txt")
no_of_sentences = 1981 # number of sentences to take for corpus

In [6]:
len(tagged_sentences)

1981

#### Separate the sentences words and tags into two different arrays

In [7]:
sentences, sentence_tags = [], []
for sent in tagged_sentences:
    sentence, lang, tags = zip(*sent)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

In [8]:
len(sentences), len(sentence_tags)

(1981, 1981)

In [9]:
sentences[0], sentence_tags[0]

(array(['I', 'will', 'read', 'kal', 'pakka', '.', 'Have', 'to', 'study',
        'right', 'now', 'okay', '?', '\xf0\x9f\x98\x80',
        'https://t.co/C2SrZhfJfK'], dtype='|S23'),
 array(['PR_PRP', 'V_VAUX', 'V_VM', 'RB_ALC', 'JJ', 'RD_PUNC', 'V_VM',
        'RP_RPD', 'N_NN', 'RB_AMN', 'RB_AMN', 'RP_INJ', 'RD_PUNC', 'E',
        'U'], dtype='|S7'))

In [10]:
len(sentences[0]), len(sentence_tags[0])

(15, 15)

### Train the required number of sentences *

In [11]:
sentences = sentences[:no_of_sentences]
sentence_tags = sentence_tags[:no_of_sentences]

In [12]:
(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [13]:
print sentences[0], sentence_tags[0]

['I' 'will' 'read' 'kal' 'pakka' '.' 'Have' 'to' 'study' 'right' 'now'
 'okay' '?' '\xf0\x9f\x98\x80' 'https://t.co/C2SrZhfJfK'] ['PR_PRP' 'V_VAUX' 'V_VM' 'RB_ALC' 'JJ' 'RD_PUNC' 'V_VM' 'RP_RPD' 'N_NN'
 'RB_AMN' 'RB_AMN' 'RP_INJ' 'RD_PUNC' 'E' 'U']


#### For the embedding layer to work, find the count of unique words

In [14]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)
        
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [15]:
print train_tags[0]

['@' 'PSP' 'RD_PUNC' 'PR_PRP' 'PSP' 'V_VM' 'V_VM' 'DT' 'V_VM' 'N_NN'
 'RD_PUNC' 'JJ' 'JJ' 'RD_PUNC' 'JJ' 'JJ' 'N_NN' 'V_VAUX' 'V_VM' 'N_NN'
 'RD_PUNC' 'E']


#### Convert the words to the numpy array so that, numerical data can be used for training

In [16]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(train_tags_y[0])
print(test_sentences_X[0])
print(test_tags_y[0])
print len(train_sentences_X[0]), len((train_tags_y[0]))

[6552, 667, 3708, 3233, 7988, 3152, 1544, 6669, 6879, 2583, 1068, 5675, 5675, 3758, 4575, 4575, 2574, 3864, 3286, 7729, 3758, 4427]
[18, 36, 32, 39, 36, 4, 4, 7, 4, 17, 32, 6, 6, 32, 6, 6, 17, 29, 4, 17, 32, 20]
[1, 7367, 2324, 3574, 673, 4370, 7015, 1038, 1850, 4727, 4727, 50, 1, 1966, 3233, 1242, 3372, 1, 6581, 8355, 343, 1294]
[18, 21, 16, 15, 36, 21, 17, 17, 17, 6, 6, 36, 20, 16, 39, 4, 7, 17, 4, 35, 32, 20]
22 22


#### Pad the sequences because Keras can only work with fixed size sequences

In [17]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)  

207


In [18]:
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[6552  667 3708 3233 7988 3152 1544 6669 6879 2583 1068 5675 5675 3758
 4575 4575 2574 3864 3286 7729 3758 4427    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [19]:
print(len(tags))

39


#### Define our neural architecture

In [20]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 207, 128)          1116544   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 207, 512)          788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 207, 40)           20520     
_________________________________________________________________
activation_1 (Activation)    (None, 207, 40)           0         
Total params: 1,925,544
Trainable params: 1,925,544
Non-trainable params: 0
_________________________________________________________________


#### Since we have 39 tags for each word we need to convert it to ONE HOT ENCODING

In [22]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [23]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print cat_train_tags_y[0]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [24]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Train on 1267 samples, validate on 317 samples
Epoch 1/40
1267/1267 [==============================] - 38s 30ms/step - loss: 1.9498 - acc: 0.8968 - ignore_accuracy: 0.0000e+00 - val_loss: 0.6445 - val_acc: 0.8982 - val_ignore_accuracy: 0.0000e+00
Epoch 2/40
1267/1267 [==============================] - 38s 30ms/step - loss: 0.4853 - acc: 0.8977 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3878 - val_acc: 0.8976 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
1267/1267 [==============================] - 35s 28ms/step - loss: 0.3636 - acc: 0.9044 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3377 - val_acc: 0.9117 - val_ignore_accuracy: 0.0000e+00
Epoch 4/40
1267/1267 [==============================] - 35s 28ms/step - loss: 0.3369 - acc: 0.9077 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3308 - val_acc: 0.9018 - val_ignore_accuracy: 0.0000e+00
Epoch 5/40
1267/1267 [==============================] - 35s 27ms/step - loss: 0.3306 - acc: 0.9024 - ignore_accuracy: 0.0000e+00 - val_loss: 0.3252 - val

In [25]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print ({model.metrics_names[1]},  {scores[1] * 100})

397/397 [==============================] - 3s 8ms/step
(set(['acc']), set([96.08780842283811]))


#### Convert back the categorical to tokens

In [26]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [36]:
predictions = model.predict(test_sentences_X)

print len(predictions[0])
print len(test_tags_y[0])

# print first few predictions
token_sequences = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
# print(logits_to_tokens(predictions[:2], {i: t for t, i in tag2index.items()}))
test_tags_sequences = logits_to_tokens(test_tags_y, {i: t for t, i in tag2index.items()})

print len(predictions), len(test_tags_sequences)
# token_sequences[0] = filter(lambda a: a != '-PAD-', token_sequences[0])
# print (test_sentences[0])
print len(token_sequences[0][:len(test_tags[0])])
print len(test_tags[0])

207
207
397 397
22
22
